In [1]:
import numpy as np
import pandas as pd
import sqlite3

import yfinance as yf

# GET DATA

In [2]:
dow_list_url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'
sp5_list_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
etf_list_url = 'https://etfdb.com/compare/volume/'

dow_list_raw = pd.read_html(dow_list_url)[1]
sp5_list_raw = pd.read_html(sp5_list_url)[0]
etf_list_raw = pd.read_html(etf_list_url)[0]

In [3]:
dow_list =  pd.DataFrame({
    'Symbol': dow_list_raw['Symbol'].str.replace('.', '-')})

sp5_list =  pd.DataFrame({
    'Symbol': sp5_list_raw['Symbol'].str.replace('.', '-'),
    'Name': sp5_list_raw['Security'],
    'Sector': sp5_list_raw['GICS Sector'],
    'Industry': sp5_list_raw['GICS Sub-Industry']})

etf_list =  pd.DataFrame({
    'Symbol': etf_list_raw['Symbol'],
    'Name': etf_list_raw['Name']})

dow_list = dow_list.merge(sp5_list)

display(dow_list)
display(sp5_list)
display(etf_list)

,Symbol,Name,Sector,Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AXP,American Express,Financials,Consumer Finance
2,AMGN,Amgen,Health Care,Biotechnology
3,AAPL,Apple,Information Technology,"Technology Hardware, Storage & Peripherals"
4,BA,Boeing,Industrials,Aerospace & Defense
5,CAT,Caterpillar,Industrials,Construction Machinery & Heavy Trucks
6,CVX,Chevron,Energy,Integrated Oil & Gas
7,CSCO,Cisco,Information Technology,Communications Equipment
8,KO,Coca-Cola,Consumer Staples,Soft Drinks
9,DIS,Disney,Communication Services,Movies & Entertainment


,Symbol,Name,Sector,Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Pharmaceuticals
4,ABMD,Abiomed,Health Care,Health Care Equipment
...,...,...,...,...
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants
500,ZBRA,Zebra,Information Technology,Electronic Equipment & Instruments
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment
502,ZION,Zions Bancorp,Financials,Regional Banks


,Symbol,Name
0,TQQQ,ProShares UltraPro QQQ
1,SPY,SPDR S&P 500 ETF Trust
2,SQQQ,ProShares UltraPro Short QQQ
3,UVXY,ProShares Ultra VIX Short-Term Futures ETF
4,QQQ,Invesco QQQ Trust
...,...,...
95,VTEB,Vanguard Tax-Exempt Bond ETF
96,ICLN,iShares Global Clean Energy ETF
97,SSO,ProShares Ultra S&P 500
98,XRT,SPDR S&P Retail ETF


In [4]:
dow_symbols = dow_list['Symbol'].values
sp5_symbols = sp5_list['Symbol'].values
etf_symbols = etf_list['Symbol'].values

symbols = np.unique(np.concatenate([
    dow_symbols,
    sp5_symbols,
    etf_symbols]))

histories_raw = yf.download(list(symbols))

[*********************100%***********************]  604 of 604 completed


In [5]:
histories = (histories_raw
    .stack()
    .reset_index()
    .rename(columns={'level_1': 'Symbol', 'Adj Close': 'Adj_Close'})
    .astype({'Date': 'datetime64[ns]', 'Volume': 'int64'}))

display(histories)

,Date,Symbol,Adj_Close,Close,High,Low,Open,Volume
0,1962-01-02,AEP,1.035685,34.312500,35.125000,34.312500,0.000000,5800
1,1962-01-02,BA,0.190931,0.823045,0.837449,0.823045,0.837449,352350
2,1962-01-02,CAT,0.495367,1.604167,1.619792,1.588542,1.604167,163200
3,1962-01-02,CNP,0.349574,10.783375,10.865333,10.783375,0.000000,13879
4,1962-01-02,CVX,0.361663,3.296131,3.296131,3.244048,0.000000,105840
...,...,...,...,...,...,...,...,...
4454003,2022-05-09,YUM,111.790001,111.790001,113.959999,111.410004,113.750000,771706
4454004,2022-05-09,ZBH,115.985001,115.985001,118.540001,115.370003,118.660004,411361
4454005,2022-05-09,ZBRA,336.160004,336.160004,341.719391,331.910004,335.000000,183611
4454006,2022-05-09,ZION,54.990002,54.990002,55.660000,54.230000,55.090000,631180


# SAVE DATA

In [6]:
# ! rm market.db
con = sqlite3.connect('market.db')

dow_list.to_sql('dow_list', con, index=False, if_exists='replace')
sp5_list.to_sql('sp5_list', con, index=False, if_exists='replace')
etf_list.to_sql('etf_list', con, index=False, if_exists='replace')

histories.to_sql('histories', con, index=False, if_exists='replace')